In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gunzip /content/drive/MyDrive/ADR/meddra_all_se.tsv.gz

In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

# بارگذاری مدل BioBERT
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# خواندن فایل
df = pd.read_csv("/content/drive/MyDrive/ADR/meddra_all_se.tsv", sep="\t", header=None,
                 names=["STITCH_CID", "UMLS_ID", "UMS_Concept_ID", "Source", "ADR_NAME"])

adr_list = df['ADR_NAME'].dropna().drop_duplicates().tolist()
print(df.shape)
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[0][0].numpy()

adr_embeddings = []
valid_adr_names = []

for adr in adr_list:
    try:
        emb = get_bert_embedding(adr)
        adr_embeddings.append(emb)
        valid_adr_names.append(adr)
    except:
        continue

# ذخیره
np.save("/content/drive/MyDrive/ADR/adr_bert_embeddings.npy", np.array(adr_embeddings))
# pd.DataFrame({'ADR': valid_adr_names}).to_csv("/content/drive/MyDrive/ADR/adr_names.csv", index=False)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(309849, 5)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ADR/meddra_all_se.tsv", sep="\t", header=None)
df.head()
adr_list
pd.DataFrame({'ADR': valid_adr_names}).to_csv("/content/drive/MyDrive/ADR/adr_names.csv", index=False)

In [ ]:
# import pandas as pd

# # خواندن فایل
# df = pd.read_csv("/content/drive/MyDrive/ADR/meddra_all_se.tsv", sep="\t", header=None,
#                  names=["STITCH_CID", "UMLS_ID", "UMS_Concept_ID", "Source", "ADR_NAME"])

# # استخراج نام ADRها
# adr_df = df[["ADR_NAME"]].drop_duplicates().reset_index(drop=True)
# adr_df.columns = ["ADR"]
# adr_df.to_csv("/content/drive/MyDrive/ADR/adr_names.csv", index=False)
# df.head()

In [ ]:
import pandas as pd

# خواندن فایل
df = pd.read_csv("/content/drive/MyDrive/ADR/meddra_all_se.tsv", sep="\t", header=None,
                 names=["STITCH_CID", "UMLS_ID", "UMS_Concept_ID", "Source", "ADR_NAME"])

# چاپ چند ردیف برای بررسی
# print(df.head())
df["CID"] = df["STITCH_CID"].str.extract(r"CID(\d+)").astype(float).astype(int)
cid_df = df[["STITCH_CID", "CID"]].drop_duplicates().reset_index(drop=True)

# ذخیره
cid_df.to_csv("/content/drive/MyDrive/ADR/drug_cids.csv", index=False)
cid_df


,STITCH_CID,CID
0,CID000010917,10917
1,CID000000119,119
2,CID000000137,137
3,CID000000143,143
4,CID000006006,6006
...,...,...
1551,CID056603655,56603655
1552,CID056842239,56842239
1553,CID070683024,70683024
1554,CID070695640,70695640


In [ ]:
!gunzip /content/drive/MyDrive/ADR/CID-SMILES.gz

^C


In [ ]:
import pandas as pd

# خواندن CIDهای مورد نظر
cid_df = pd.read_csv("/content/drive/MyDrive/ADR/drug_cids.csv")
# تبدیل به رشته و حذف صفرهای سمت چپ
our_cids = set(cid_df["CID"].astype(str).str.lstrip('0'))

# link to CID-SMILES.gz file: https://drive.google.com/file/d/1DfgWLqc2AUupA5wPYV__O9umZG03KMDX/view?usp=drive_link
# OR: https://ftp.ncbi.nlm.nih.gov/pubchem/Compound/Extras/

chunks = pd.read_csv("/content/drive/MyDrive/ADR/CID-SMILES.gz", sep="\t", header=None, names=["CID", "SMILES"], compression='gzip', chunksize=100000)

all_data = []
for chunk in chunks:
    # پاک‌سازی ستون CID در chunk (رشته + حذف صفر پیشوند)
    chunk["CID"] = chunk["CID"].astype(str).str.lstrip('0')
    filtered = chunk[chunk["CID"].isin(our_cids)]
    all_data.append(filtered)

if all_data:
    drug_smiles_df = pd.concat(all_data)
else:
    drug_smiles_df = pd.DataFrame(columns=["CID", "SMILES"])

drug_smiles_df.to_csv("/content/drive/MyDrive/ADR/drug_smiles.csv", index=False)

print(f"Found {len(drug_smiles_df)} SMILES for given CIDs.")



Found 1505 SMILES for given CIDs.


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ADR/CID-SMILES.gz", sep="\t", header=None, names=["CID", "SMILES"], compression="gzip", nrows=10)
print(df)


   CID                                     SMILES
0    1           CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C
1    2              CC(=O)OC(CC(=O)O)C[N+](C)(C)C
2    3                   C1=CC(C(C(=C1)C(=O)O)O)O
3    4                                    CC(CN)O
4    5                       C(C(=O)COP(=O)(O)O)N
5    6  C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl
6    7                     CCN1C=NC2=C(N=CN=C21)N
7    8                        CCC(C)(C(C(=O)O)O)O
8    9          C1(C(C(C(C(C1O)O)OP(=O)(O)O)O)O)O
9   11                                   C(CCl)Cl


In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import numpy as np

# فرض: SMILES strings in 'drug_smiles.csv' with column 'SMILES'
smiles_df = pd.read_csv("/content/drive/MyDrive/ADR/drug_smiles.csv")
tokenizer = RobertaTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model = RobertaModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

drug_embeddings = []

for text in smiles_df["SMILES"]:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        emb = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        drug_embeddings.append(emb)

drug_embeddings = np.array(drug_embeddings)  # (n_drug, 768)
np.save("/content/drive/MyDrive/ADR/drug_chemberta_embeddings.npy", drug_embeddings)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
